In [4]:
#!pip install pymysql
import boto3
from configparser import ConfigParser
import pandas as pd
import pymysql
from io import BytesIO

In [5]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("../config/config.ini")

['../config/config.ini']

In [6]:
# Configurações para conexão com o MinIO/S3
endpoint = config.get("MinIO", "endpoint")
access_key = config.get("MinIO", "access_key")
secret_key = config.get("MinIO", "secret_key")
bucket_trust = config.get("Bucket", "bucket_trust")

# Nome do arquivo a ser lido
source_filename = config.get("FILE", "censo_saude_csv")

In [7]:
print("endpoint: ", endpoint)
print("access_key: ", access_key)
print("secret_key: ", secret_key)
print("bucket_trust: ", bucket_trust)
print("source_filename: ", source_filename)

endpoint:  http://minio:9000
access_key:  aulafia
secret_key:  aulafia@123
bucket_trust:  trust
source_filename:  CensoSaude.csv


In [8]:
# Inicializar o cliente boto3 para S3
minio_client = boto3.client("s3", 
                            endpoint_url=endpoint,
                            aws_access_key_id=access_key,
                            aws_secret_access_key=secret_key
)

In [9]:
# Obter o conteúdo do arquivo CSV do MinIO
response = minio_client.get_object(Bucket=bucket_trust, Key=source_filename)
csv_content = response['Body'].read()

In [10]:
# Ler o conteúdo do CSV em um DataFrame
data_frame = pd.read_csv(BytesIO(csv_content))

In [ ]:
# Conectar ao MySQL
mysql_connection = pymysql.connect(host=mysql_host,
                                   user=mysql_user,
                                   password=mysql_password,
                                   db=mysql_db,
                                   cursorclass=pymysql.cursors.DictCursor)

try:
    # Apagar a tabela coordenadas (se existir)
    with mysql_connection.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS coordenadas")

    # Criar novamente a tabela coordenadas
    with mysql_connection.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE coordenadas (
                codigo_ibge INT NOT NULL,
                nome VARCHAR(100) NOT NULL,
                latitude FLOAT(8) NOT NULL,
                longitude FLOAT(8) NOT NULL,
                capital BOOLEAN NOT NULL,
                codigo_uf INT NOT NULL,
                siafi_id VARCHAR(4) NOT NULL UNIQUE,
                ddd INT NOT NULL,
                fuso_horario VARCHAR(32) NOT NULL,
                PRIMARY KEY (codigo_ibge)
            )
        """)
    
    # Inserir os dados na tabela coordenadas
    with mysql_connection.cursor() as cursor:
        for index, row in data_frame.iterrows():
            sql = "INSERT INTO coordenadas VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            values = (row['codigo_ibge'], row['nome'], row['latitude'], row['longitude'],
                      row['capital'], row['codigo_uf'], row['siafi_id'], row['ddd'], row['fuso_horario'])
            cursor.execute(sql, values)
        mysql_connection.commit()

    print(f'Dados do arquivo {csv_filename} inseridos na tabela coordenadas com sucesso.')

finally:
    mysql_connection.close()

In [ ]:
# Criar um fluxo de gravação em buffer para o novo arquivo CSV
output_buffer = io.StringIO()

In [ ]:
# Salvar o DataFrame reduzido como um arquivo CSV no buffer
selected_columns.to_csv(output_buffer, index=False, encoding='utf-8')

In [ ]:
# Obter o conteúdo do buffer como bytes codificados em UTF-8
encoded_csv_content = output_buffer.getvalue().encode('utf-8')

In [ ]:
# Enviar o novo arquivo CSV codificado para o bucket de destino
minio_client.put_object(Bucket=bucket_trust, Key=target_filename, Body=encoded_csv_content)